In [1]:
import torch; torch.manual_seed(0)
import torch.nn as nn
import torch.nn.functional as F
import torch.utils
import torch.distributions
import torchvision
import numpy as np
import pandas as pd
import torchvision.models as models 
from torchvision import transforms, utils
from torch.utils.data.sampler import SubsetRandomSampler
import torch.optim as optim
import matplotlib.pyplot as plt
import latent_plot as lp
from torch.autograd import Variable
from torch.optim import lr_scheduler
import seaborn as sns
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
import copy
#from model import VAE

import matplotlib.pyplot as plt; plt.rcParams['figure.dpi'] = 200
from data_loader import *
from model import convVAE
from sklearn.manifold import TSNE

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
csv_path = 'oct_gr.csv'
img_path = '/home/vip/sayan-mandal/datasets/obj_criteria/good_reduced/'

ResizeParam = 128
FD = LSTMPrepDataLoader(csv_path, img_path,transform=transforms.Compose([transforms.Resize(ResizeParam), transforms.ToTensor()]))

#random sampler
batch_size = 64


data_loader = torch.utils.data.DataLoader(FD, batch_size=batch_size, shuffle = True)

In [4]:
#vae model structure
nl = 128
model = convVAE((3,128,128),nl).cuda()
model.load_state_dict(torch.load('/home/vip/sayan-mandal/vae-convmodels/dfcvae_inp128_nl'+str(nl)+'_a1b1.torch'))
print(model)

convVAE(
  (encoder): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): LeakyReLU(negative_slope=0.01)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): LeakyReLU(negative_slope=0.01)
    (9): Conv2d(64, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (10): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): LeakyReLU(negative_slope=0.01)
    (12): Flatten(start_dim=1, end_dim=-1)
  )
  (fc1): Sequential(
    (0): Linear(in_features=16384, out_features=4096, bias=True)
    (1): BatchNo

In [5]:
for i, (data,me,age,pdays,l2) in enumerate(data_loader):
    y, _,_ =  model.encode(Variable(data).cuda())
    y = y.cpu().detach().numpy()
    if i == 0:
        yd = y
        med = np.array(me)
        aged = np.array(age)
        pdaysd = np.array(pdays)
        lbl2 = np.array(l2)
    else:
        yd = np.append(yd,y,axis=0)
        med = np.append(med,np.array(me), axis=0)
        aged = np.append(aged,np.array(age), axis=0)
        pdaysd = np.append(pdaysd,np.array(pdays), axis=0)
        lbl2= np.append(lbl2,np.array(l2), axis=0)


In [6]:
med = med.reshape(-1,1)
aged = aged.reshape(-1,1)
pdaysd = pdaysd.reshape(-1,1)
lbl2 = lbl2.reshape(-1,1)

In [7]:
cols = ['maskedeye','day','age'] + ['lf_'+str(i) for i in range(1,yd.shape[1]+1)]+['label']


In [8]:
to_df = np.concatenate((med,pdaysd,aged,yd,lbl2),axis=1)
df = pd.DataFrame(data=to_df, columns=cols)

In [9]:
df[df['label']=='p']

,maskedeye,day,age,lf_1,lf_2,lf_3,lf_4,lf_5,lf_6,lf_7,...,lf_120,lf_121,lf_122,lf_123,lf_124,lf_125,lf_126,lf_127,lf_128,label
13,ny7414l,3578,69.73579739904176,-0.045665696,-0.048671898,0.022064961,-0.079083346,0.07084835,0.06965456,0.059679437,...,-0.08126251,-0.04377546,0.0023176996,0.014653523,0.08083493,0.097888194,0.16218565,-0.10002147,0.047445714,p
19,ny7442r,3178,64.82409308692677,0.01331821,0.06688894,0.00081636943,-0.09387414,-0.030329728,-0.038092077,0.0031380772,...,-0.044374403,-0.0017060178,0.06695573,-0.089685865,-0.05006759,-0.008818753,-0.04589654,0.10608152,0.068900354,p
23,sd4180l,1344,61.073237508555785,-0.05647586,-0.039923903,-0.042601988,-0.016306728,0.058632318,-0.026276087,0.05485032,...,-0.02629549,0.039044566,-0.0482104,0.02326956,-0.022188853,-0.023054507,-0.014828452,0.0283194,0.08086174,p
24,sd4207r,1179,69.1690622861054,0.03675268,-0.08729698,0.082590856,-0.036059774,-0.030947777,-0.009508303,0.054328278,...,0.059173334,-0.15904649,0.046931546,-0.03572343,-0.15311995,-0.016857762,-0.022206943,0.09485433,0.010220545,p
32,ny7809r,3489,62.65845311430527,-0.45507625,-0.24729033,-0.52965486,0.48810542,0.45205057,0.36695707,-0.5821227,...,0.19661026,-0.26063636,-0.1028435,-0.13547885,-0.051771574,-0.20192504,-0.062104322,0.104844145,-0.10019231,p
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15653,sd3072r,2481,55.4113620807666,-0.010292013,-0.022177873,-0.16056895,-0.10765521,0.0067940564,0.027162518,-0.00915597,...,-0.02497689,-0.07608875,-0.0025844667,0.010021769,-0.055373788,-0.1455486,-0.075136445,-0.03278825,-0.1540613,p
15679,sd1239l,7653,69.19370294318959,-0.03282065,0.11201097,-0.06229058,0.06394042,0.01682323,-0.09495189,-0.041555073,...,0.04424555,-0.038828604,-0.0005449027,0.030699218,0.05337616,0.020547943,-0.015645208,-0.008011344,0.045455668,p
15681,sd1172l,5041,67.95893223819301,-0.16035001,0.03529792,-0.0048728525,-0.052761804,0.23064107,0.15642215,0.01436406,...,-0.0738098,-0.26337865,-0.08750463,-0.1164832,0.12586166,0.049023475,0.15981317,-0.0105161425,0.02616854,p
15685,sd2620r,3625,83.49349760438056,0.16046381,0.061628282,0.022864131,-0.0657341,0.020726966,-0.032090794,0.15159291,...,-0.16514565,-0.13459569,-0.04316032,-0.035285015,0.059595305,0.021654408,-0.1227898,0.0094832815,-0.048207045,p


In [10]:
df.to_csv('goodreduced-vae.csv',index=False)

In [16]:
df[df['maskedeye'] == 'sd3568l']

,maskedeye,day,age,lf_1,lf_2,lf_3,lf_4,lf_5,lf_6,lf_7,...,lf_120,lf_121,lf_122,lf_123,lf_124,lf_125,lf_126,lf_127,lf_128,label
3278,sd3568l,2251,71.33196440793976,0.14298773,0.0060276818,0.015516873,0.00899219,-0.07919139,0.05208549,-0.017493363,...,-0.0114014475,0.031837806,0.037849773,0.040480442,-0.0029101819,0.07179062,0.16835557,-0.059593495,0.013685477,p
4711,sd3568l,830,67.44147843942505,-0.08532517,0.06551425,-0.0052915313,-0.08487617,0.02622897,0.009027556,0.04133585,...,-0.039834198,0.04582874,0.025605902,0.031950153,0.07639266,0.055034492,0.013454972,0.00517792,-0.026394311,n
13064,sd3568l,1064,68.08213552361396,-0.1219448,-0.011621111,-0.11667099,-0.11673275,-0.026674172,0.08111908,-0.03598609,...,-0.12335333,0.07710719,0.09221715,0.038620546,0.03179641,0.1828904,0.2328347,-0.18774876,-0.11133579,n
13869,sd3568l,2604,72.2984257357974,-0.11546578,0.022660105,-0.03224034,-0.056736328,-0.032242414,-0.013178927,0.06630129,...,-0.0870233,0.06557695,0.09025063,0.03470961,0.09355721,0.053269275,0.052717317,-0.046395045,-0.011355078,p
22000,sd3568l,1855,70.24777549623545,0.023510452,0.07112806,-0.02287579,-0.08158882,0.017531084,0.053396273,0.024995925,...,-0.0004606084,0.033237223,0.014196183,0.007552829,0.075150244,0.088192195,0.12571675,-0.065147385,-0.033147886,p
